In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [2]:
spark = SparkSession.builder.appName("spark_session").config("spark.jars", "C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()

In [3]:
url = "jdbc:mysql://localhost:3306/extenso_assignment"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.jdbc.Driver"
}

In [4]:
def table(table_name):
    df = spark.read.jdbc(url=url, table=table_name, properties=properties)
    return df

In [5]:
customer_profile = table('customer_profile')
product_category = table("product_category")
product_category_map = table("product_category_map")
products = table("products")
rw_transaction_data = table("rw_transaction_data")

In [6]:
customer_profile.show()

+----------+------------+-------------------+--------------------+-----------------------+------------------+-------------------+----------------------+---------------------+----------------------+-------------------------+--------------------+---------------------+------------------------+-------------------+----------------+------------------+--------------------------------+-------------+-------------+-----------------+------------------------+-----------------------+----------+
|account_id|reward_point|total_inflow_amount|total_outflow_amount|total_valuechain_amount|total_inflow_count|total_outflow_count|total_valuechain_count|monthly_inflow_amount|monthly_outflow_amount|monthly_valuechain_amount|monthly_inflow_count|monthly_outflow_count|monthly_valuechain_count|latest_used_product|latest_tran_date|this_month_revenue|monthly_average_lifetime_revenue|total_revenue|product_usage|most_used_product|second_most_used_product|third_most_used_product|  run_date|
+----------+------------+-

In [7]:
rw_transaction_data.printSchema()

root
 |-- txn_id: long (nullable = true)
 |-- last_modified_date: date (nullable = true)
 |-- last_modified_date_bs: string (nullable = true)
 |-- created_date: date (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: integer (nullable = true)
 |-- module_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- payer_account_id: integer (nullable = true)
 |-- receiver_account_id: integer (nullable = true)
 |-- reward_point: double (nullable = true)
 |-- cash_back_amount: double (nullable = true)
 |-- revenue_amount: double (nullable = true)
 |-- transactor_module_id: long (nullable = true)
 |-- time: string (nullable = true)



In [8]:
rw_transaction_data = rw_transaction_data.withColumn("months",month(col("last_modified_date")))
rw_transaction_data = rw_transaction_data.withColumn("last_modified_date", col("last_modified_date").cast("string"))

In [9]:
rw_transaction_data.printSchema()

root
 |-- txn_id: long (nullable = true)
 |-- last_modified_date: string (nullable = true)
 |-- last_modified_date_bs: string (nullable = true)
 |-- created_date: date (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: integer (nullable = true)
 |-- module_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- payer_account_id: integer (nullable = true)
 |-- receiver_account_id: integer (nullable = true)
 |-- reward_point: double (nullable = true)
 |-- cash_back_amount: double (nullable = true)
 |-- revenue_amount: double (nullable = true)
 |-- transactor_module_id: long (nullable = true)
 |-- time: string (nullable = true)
 |-- months: integer (nullable = true)



In [10]:
rw_transaction_data.show()

+---------+------------------+---------------------+------------+------+------+---------+----------+---------------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+
|   txn_id|last_modified_date|last_modified_date_bs|created_date|amount|status|module_id|product_id|product_type_id|payer_account_id|receiver_account_id|reward_point|cash_back_amount|revenue_amount|transactor_module_id|    time|months|
+---------+------------------+---------------------+------------+------+------+---------+----------+---------------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+
|660612529|        2023-03-29|           2079-12-15|  2022-11-23|  50.0|     1|        1|        77|             29|             531|                  2|         0.0|             0.0|           0.0|                   4|14:07:40|     3|
|666435422|        2022-12-01|           2079-08-15|  20

In [11]:
rw_transaction_data.select("payer_account_id").distinct().show()

+----------------+
|payer_account_id|
+----------------+
|             471|
|              34|
|              26|
|            1056|
|             222|
|             531|
|              56|
+----------------+



In [12]:
joined = rw_transaction_data.join(product_category_map,['product_id', 'product_type_id','module_id'])

In [13]:
joined.show()

+----------+---------------+---------+---------+------------------+---------------------+------------+------+------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+--------------------+-------------------+-----------+
|product_id|product_type_id|module_id|   txn_id|last_modified_date|last_modified_date_bs|created_date|amount|status|payer_account_id|receiver_account_id|reward_point|cash_back_amount|revenue_amount|transactor_module_id|    time|months|        product_name|product_category_id|   txn_flow|
+----------+---------------+---------+---------+------------------+---------------------+------------+------+------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+--------------------+-------------------+-----------+
|       143|             59|        1|693893736|        2023-01-06|           2079-09-22|  2023-01-06| 175.0|     1|             531|

In [14]:
joined_total = joined.groupBy(["payer_account_id","months"]).pivot("txn_flow").sum("amount").fillna(0)

In [15]:
joined_total.show()

+----------------+------+---------+---------+-----------+
|payer_account_id|months|   InFlow|  OutFlow|Value Chain|
+----------------+------+---------+---------+-----------+
|             471|     1|    700.0|    700.0|    18850.0|
|              34|    12|  83200.0|  88200.0|     4351.0|
|              26|    12|      0.0|      0.0|      460.0|
|              26|     1|  34340.0|  51340.0|    18011.0|
|             222|    12|      0.0|      0.0|      270.0|
|            1056|    12|      0.0|      0.0|      400.0|
|             471|    12|      0.0|      0.0|     3048.0|
|             222|     1|      0.0|      0.0|      240.0|
|             531|    12|      0.0|      0.0|   600054.0|
|              34|     1|2133150.0|2396770.0|   246737.0|
|              56|    12|   9725.0|  12125.0|    18301.0|
|             531|     1|      0.0|      0.0|  1275093.0|
|             531|     3|      0.0|      0.0|      100.0|
|              56|     1|  15750.0| 135750.0|    68204.0|
+-------------

In [16]:
joined_count = joined.groupBy(["payer_account_id","months"]).pivot("txn_flow").count().fillna(0)

In [17]:
joined_count.show()

+----------------+------+------+-------+-----------+
|payer_account_id|months|InFlow|OutFlow|Value Chain|
+----------------+------+------+-------+-----------+
|             471|     1|     2|      2|         96|
|              34|    12|     5|      6|          6|
|              26|    12|     0|      0|         11|
|              26|     1|    12|     15|         42|
|             222|    12|     0|      0|          2|
|            1056|    12|     0|      0|          2|
|             471|    12|     0|      0|         20|
|             222|     1|     0|      0|          2|
|             531|    12|     0|      0|       3836|
|              34|     1|   121|    137|         74|
|              56|    12|     3|      4|         40|
|             531|     1|     0|      0|       6157|
|             531|     3|     0|      0|          2|
|              56|     1|    12|     18|        153|
+----------------+------+------+-------+-----------+



In [18]:
total_count = joined_count.groupBy("payer_account_id").agg(sum("InFlow").alias("TotalCountInflow"),sum("Outflow").alias("TotalCountOutflow"),sum("Value Chain").alias("TotalCountValueChain"))

In [19]:
total_amount = joined_total.groupBy("payer_account_id").agg(sum("InFlow").alias("TotalAmtInflow"),sum("Outflow").alias("TotalAmtOutflow"),sum("Value Chain").alias("TotalAmtValueChain"))

In [20]:
total_amount.show()

+----------------+--------------+---------------+------------------+
|payer_account_id|TotalAmtInflow|TotalAmtOutflow|TotalAmtValueChain|
+----------------+--------------+---------------+------------------+
|             471|         700.0|          700.0|           21898.0|
|              34|     2216350.0|      2484970.0|          251088.0|
|              26|       34340.0|        51340.0|           18471.0|
|            1056|           0.0|            0.0|             400.0|
|             222|           0.0|            0.0|             510.0|
|             531|           0.0|            0.0|         1875247.0|
|              56|       25475.0|       147875.0|           86505.0|
+----------------+--------------+---------------+------------------+



In [21]:
avg_count = joined_count.groupBy("payer_account_id").agg(avg("InFlow").alias("AvgCountInflow"),avg("Outflow").alias("AvgCountOutflow"),avg("Value Chain").alias("AvgCountValueChain"))

In [22]:
avg_count.show()

+----------------+--------------+---------------+------------------+
|payer_account_id|AvgCountInflow|AvgCountOutflow|AvgCountValueChain|
+----------------+--------------+---------------+------------------+
|             471|           1.0|            1.0|              58.0|
|              34|          63.0|           71.5|              40.0|
|              26|           6.0|            7.5|              26.5|
|            1056|           0.0|            0.0|               2.0|
|             222|           0.0|            0.0|               2.0|
|             531|           0.0|            0.0|3331.6666666666665|
|              56|           7.5|           11.0|              96.5|
+----------------+--------------+---------------+------------------+



In [23]:
avg_total = joined_total.groupBy("payer_account_id").agg(avg("InFlow").alias("AvgAmtInflow"),avg("Outflow").alias("AvgAmtOutflow"),avg("Value Chain").alias("AvgAmtValueChain"))

In [24]:
avg_total.show()

+----------------+------------+-------------+-----------------+
|payer_account_id|AvgAmtInflow|AvgAmtOutflow| AvgAmtValueChain|
+----------------+------------+-------------+-----------------+
|             471|       350.0|        350.0|          10949.0|
|              34|   1108175.0|    1242485.0|         125544.0|
|              26|     17170.0|      25670.0|           9235.5|
|            1056|         0.0|          0.0|            400.0|
|             222|         0.0|          0.0|            255.0|
|             531|         0.0|          0.0|625082.3333333334|
|              56|     12737.5|      73937.5|          43252.5|
+----------------+------------+-------------+-----------------+



In [25]:
reward_point = joined.groupBy("payer_account_id").agg(sum("reward_point").alias("TotalRewardPoint"))

In [26]:
reward_point.show()

+----------------+----------------+
|payer_account_id|TotalRewardPoint|
+----------------+----------------+
|             471|            14.0|
|              34|            68.0|
|              26|             0.0|
|            1056|             8.0|
|             222|             0.0|
|             531|             0.0|
|              56|            39.0|
+----------------+----------------+



In [27]:
joined = joined.withColumn("Date", concat_ws(" ", col("last_modified_date"), col("time")))

In [28]:
joined.show()

+----------+---------------+---------+---------+------------------+---------------------+------------+------+------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+--------------------+-------------------+-----------+-------------------+
|product_id|product_type_id|module_id|   txn_id|last_modified_date|last_modified_date_bs|created_date|amount|status|payer_account_id|receiver_account_id|reward_point|cash_back_amount|revenue_amount|transactor_module_id|    time|months|        product_name|product_category_id|   txn_flow|               Date|
+----------+---------------+---------+---------+------------------+---------------------+------------+------+------+----------------+-------------------+------------+----------------+--------------+--------------------+--------+------+--------------------+-------------------+-----------+-------------------+
|       143|             59|        1|693893736|        2023-01-06|      

In [29]:
joined = joined.withColumn("Date", to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss"))

In [30]:
# joined.show()

In [31]:
last_transaction = joined.groupBy("payer_account_id").agg(max("Date").alias("Latest_Transaction_Date"))

In [32]:
joined_alias = joined.alias("joined")

In [33]:
joined_and_last = joined_alias.join(
    last_transaction,
    last_transaction["Latest_Transaction_Date"] == joined_alias["Date"],how="semi"
).select(joined_alias["payer_account_id"],joined_alias["Date"].alias("Latest_Transaction_Date"),joined_alias["product_name"].alias("latest_Product"))
joined_and_last = joined_and_last.dropDuplicates(['payer_account_id'])

In [34]:
joined_and_last.show()

+----------------+-----------------------+--------------------+
|payer_account_id|Latest_Transaction_Date|      latest_Product|
+----------------+-----------------------+--------------------+
|              26|    2023-01-06 11:13:47|    NT Prepaid Topup|
|              34|    2023-01-06 11:34:40|         Electricity|
|              56|    2023-01-06 11:36:30|         Electricity|
|             222|    2023-01-01 09:39:35|         Ncell Topup|
|             471|    2023-01-06 11:17:50|    NT Prepaid Topup|
|             531|    2023-03-29 14:07:40|Ncell Topup via Bank|
|            1056|    2022-12-01 21:04:02|    NT Prepaid Topup|
+----------------+-----------------------+--------------------+



In [35]:
rev_amt = joined.groupBy("payer_account_id","months").agg(sum("revenue_amount").alias("rev_amt"))                                                         

In [36]:
rev_amt.show()

+----------------+------+------------------+
|payer_account_id|months|           rev_amt|
+----------------+------+------------------+
|             471|     1|  88.3400000000001|
|              34|    12|              17.2|
|              26|    12|0.9299999999999993|
|              26|     1| 39.92999999999998|
|             222|    12|2.0199999999999996|
|            1056|    12|              14.0|
|             471|    12|11.439999999999998|
|             222|     1|1.7999999999999998|
|             531|    12|            505.81|
|              34|     1|499.23000000000013|
|              56|    12|24.999999999999986|
|             531|     1|1768.2299999999996|
|             531|     3|               0.0|
|              56|     1|            177.11|
+----------------+------+------------------+



In [37]:
rev_amt_tot_mean = rev_amt.groupBy("payer_account_id").agg(sum("rev_amt").alias("Total_Revenue_Amount"),mean("rev_amt").alias("Monthly_Revenue_Amount"))

In [38]:
rev_amt_tot_mean.show()

+----------------+--------------------+----------------------+
|payer_account_id|Total_Revenue_Amount|Monthly_Revenue_Amount|
+----------------+--------------------+----------------------+
|             471|    99.7800000000001|     49.89000000000005|
|              34|   516.4300000000002|     258.2150000000001|
|              26|   40.85999999999998|     20.42999999999999|
|            1056|                14.0|                  14.0|
|             222|  3.8199999999999994|    1.9099999999999997|
|             531|  2274.0399999999995|     758.0133333333332|
|              56|              202.11|               101.055|
+----------------+--------------------+----------------------+



In [40]:
product_used_count = joined.groupBy("payer_account_id","product_name").count()

In [41]:
product_used_count.show()

+----------------+--------------------+-----+
|payer_account_id|        product_name|count|
+----------------+--------------------+-----+
|              34|          Send Money|  248|
|             531| Budhabare Khanepani|    1|
|             531|   Bhaluhi khanepani|    3|
|              34|           Prabhu TV|    1|
|             531|Khanepani Sanstha...|   21|
|             531|    Brihat Khanepani|    7|
|              34|Chhimek Laghubitt...|    1|
|             531| Besisahar Khanepani|    8|
|              56|eSewa to Citizens...|    5|
|              56|          Ncell Pack|   16|
|             471|          Send Money|    4|
|              26|             NT FTTH|    1|
|             531|   Katunje Khanepani|    1|
|             531| Dhulikhel Khanepani|    1|
|              26|      eScrow Service|    2|
|             531|Khanepani Sanstha...|   11|
|             531|   Surunga Khanepani|    1|
|             531|  Godawari Khanepani|    1|
|              56|         Electri

In [42]:
window_spec = Window.partitionBy("payer_account_id").orderBy(col("count").desc())

In [43]:
data = product_used_count.orderBy("payer_account_id","count",ascending=[0, 0])

In [44]:
data.show()

+----------------+--------------------+-----+
|payer_account_id|        product_name|count|
+----------------+--------------------+-----+
|            1056|       Prepaid Topup|    1|
|            1056|    NT Prepaid Topup|    1|
|             531|   NT Topup via Bank| 6495|
|             531|TOPUP VIA BANK DI...| 1361|
|             531|Ncell Topup via Bank| 1361|
|             531| Ncell Data Via Bank|  172|
|             531|Wordlink Topup vi...|   87|
|             531| Gaindakot Khanepani|   81|
|             531|Shankarnagar Khan...|   28|
|             531|  Lekhnath Khanepani|   25|
|             531|   Devdaha Khanepani|   25|
|             531|  Kawasoti Khanepani|   24|
|             531|Khanepani Sanstha...|   21|
|             531|Mukundapur Khanepani|   21|
|             531|Nepal Telecom Dat...|   20|
|             531|Jaluke Water Supp...|   17|
|             531|   Itahari Khanepani|   16|
|             531|Pragatinagar Khan...|   16|
|             531|Karahiya Khanepa

In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [46]:
datas_count_sorted_with_row_num = data.withColumn("row_num", row_number().over(window_spec))


In [47]:
datas_count_sorted_with_row_num.show()

+----------------+--------------------+-----+-------+
|payer_account_id|        product_name|count|row_num|
+----------------+--------------------+-----+-------+
|              26|    NT Prepaid Topup|   18|      1|
|              26|       Prepaid Topup|   18|      2|
|              26|          Send Money|   12|      3|
|              26|             Cash In|   10|      4|
|              26|         Ncell Topup|   10|      5|
|              26|      eScrow Service|    2|      6|
|              26|     Fonepay Payment|    2|      7|
|              26|             NT FTTH|    1|      8|
|              26|      Postpaid Topup|    1|      9|
|              26|    Smart Cell Topup|    1|     10|
|              26|           WorldLink|    1|     11|
|              26|eSewa to Mahalaxm...|    1|     12|
|              26|eSewa to Laxmi Su...|    1|     13|
|              26|   NT Postpaid Topup|    1|     14|
|              26|eSewa to Everest ...|    1|     15|
|              34|          

In [48]:
most_used_product = datas_count_sorted_with_row_num.filter(col("row_num") == 1).select("payer_account_id", col("product_name").alias("most_used_product"))
second_used_product = datas_count_sorted_with_row_num.filter(col("row_num") == 2).select("payer_account_id", col("product_name").alias("Second_most_used_product"))
third_used_product = datas_count_sorted_with_row_num.filter(col("row_num") == 3).select("payer_account_id", col("product_name").alias("Third_most_used_product"))
product_usage = joined.groupBy("payer_account_id").count()

In [49]:
final_table = total_amount.join(avg_total,['payer_account_id'],how="left").join(total_count,['payer_account_id'],how="left").join(avg_count,['payer_account_id'],how="left").join(joined_and_last,['payer_account_id'],how="left").join(rev_amt_tot_mean,['payer_account_id'],how="left").join(product_usage,['payer_account_id'],how="left").join(most_used_product,['payer_account_id'],how="left").join(second_used_product,['payer_account_id'],how="left").join(third_used_product,['payer_account_id'],how="left")

In [50]:
final_table.show()

+----------------+--------------+---------------+------------------+------------+-------------+-----------------+----------------+-----------------+--------------------+--------------+---------------+------------------+-----------------------+--------------------+--------------------+----------------------+-----+-----------------+------------------------+-----------------------+
|payer_account_id|TotalAmtInflow|TotalAmtOutflow|TotalAmtValueChain|AvgAmtInflow|AvgAmtOutflow| AvgAmtValueChain|TotalCountInflow|TotalCountOutflow|TotalCountValueChain|AvgCountInflow|AvgCountOutflow|AvgCountValueChain|Latest_Transaction_Date|      latest_Product|Total_Revenue_Amount|Monthly_Revenue_Amount|count|most_used_product|Second_most_used_product|Third_most_used_product|
+----------------+--------------+---------------+------------------+------------+-------------+-----------------+----------------+-----------------+--------------------+--------------+---------------+------------------+-----------------